In [1]:
# nom des chemins d'entrée
chemin = 'donnees_entree/'
nomFichierReferentiel = chemin + 'referentielPuiseux.csv'
nomFichierReserve = chemin + 'Rapport stocks réserves.xlsx'
nomFichierOccupationReserve = chemin + 'SCI-RS028-OCCUPATION-RESERVES.xlsx'
nomFichierRapportDeselection = chemin + "SCI-RPP004-GP41_Rapport désélection vague - Email automatique.xlsx"
NomFichierCauseActeurDeselection = chemin + "cause_acteur_deselec.csv"
NomFichierstockactif=chemin + "SCI-RS017-STOCK-ACTIFS.xlsx"

# Nom du fichier de sortie PDF 
from datetime import datetime
aujourdhui = datetime.today().strftime('%y_%m_%d')
chemin_sortie_pdf = "donnees_sortie_pdf/"+ aujourdhui+"_fiche_staff.pdf"

In [2]:
# importation de quelques librairies utiles

import os # pour naviguer dans l'arborescence
import datetime as dt # module pour avoir les date

from math import * #  module pour avoir quelques fonctions mathématiques
import matplotlib.pyplot as plt # pour pouvoir tracer des graphes
import pandas as pd # pour pouvoir manipuler de la donnée
import numpy as np # pour faire du calcul scientifique
import chardet # permet de lire correctement un fichier

# module permettant d'importer correctement les fichiers, créé sur mesure
import module_importation as importation  

import importlib # permet de recharger un module
importlib.reload(importation)


from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer, KeepTogether
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet
import pandas as pd
from reportlab.lib.pagesizes import letter
from reportlab.lib import colors
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.platypus import PageBreak

In [3]:
#importer le rapport référentiel puiseux
referentielPuiseux = importation.fonction_importation_globale(nomFichierReferentiel, type='referentiel')

In [4]:
#importer le rapport stock réserve
rapport_stock_reserve = importation.fonction_importation_globale(nomFichierReserve)

In [5]:
#importer le rapport taux d'occupation
occupation_reserve= importation.fonction_importation_globale(nomFichierOccupationReserve, sheet_name="Détail Emplacements_2")

In [6]:
#importer le rapport déselection et les causes/acteurs de désélection
rapport_deselec = importation.fonction_importation_globale(nomFichierRapportDeselection, skiprows=2)
nombre_lignes_non_vaguees = importation.fonction_importation_globale(nomFichierRapportDeselection)
cause_acteur_deselec = importation.fonction_importation_globale(NomFichierCauseActeurDeselection)

In [7]:
#importer le rapport stock actif
rapport_stock_actif = importation.fonction_importation_globale(NomFichierstockactif)

In [8]:
# données de départ

liste_contenant = rapport_stock_reserve['Contenant'].unique().tolist()
liste_aire = rapport_stock_reserve['Aire'].unique().tolist()
liste_contenant_interessant = ['P10', 'P40', 'P20', 'S76', 'M12', 'M30', 'B60']
liste_aire_interessante= ['35', '67', '11', '89', '12']
liste_cellule_interessante= ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11']
liste_contenant_assimilé_P40 = ['P40', 'P20', 'S76']
liste_taille_emplacement_interessant = ['125', '130', '135', '140','145', '150','160','165']
correspondance_taille_nom_emplacement=pd.DataFrame({"Taille d'emplacement": liste_taille_emplacement_interessant,
        'Nom': ["P40", "P10", "P40", "B60","M12","M30","P40","P40"]})
correspondance_taille_nom_emplacement["Taille d'emplacement"] = correspondance_taille_nom_emplacement["Taille d'emplacement"].astype(str)

correspondance_taille_nom_emplacement


,Taille d'emplacement,Nom
0,125,P40
1,130,P10
2,135,P40
3,140,B60
4,145,M12
5,150,M30
6,160,P40
7,165,P40


In [9]:
print("\n KPI 1 : Volume moyen en litre par type de contenant et par aire des 100 ilpn les moins denses")

resultatKPI1 = pd.DataFrame(0, index=liste_cellule_interessante, columns=liste_contenant_interessant)
resultat_compte = pd.DataFrame(0, index=liste_cellule_interessante, columns=liste_contenant_interessant)

for contenant in liste_contenant_interessant :
    for cellule in liste_cellule_interessante :
        liste = (rapport_stock_reserve[(rapport_stock_reserve['Zone'] == cellule) & (rapport_stock_reserve["Contenant"] == contenant)]["m3"].values).tolist()
        liste=sorted(liste)
        liste = list(filter(lambda x: x != 0, liste))
        if len(liste) > 100 :
            liste = liste[0:100]
        moyenne = sum(liste)*1000 / max(len(liste), 1)
        resultatKPI1.loc[cellule, contenant] = moyenne

resultatKPI1=np.round(resultatKPI1,2)
resultatKPI1


 KPI 1 : Volume moyen en litre par type de contenant et par aire des 100 ilpn les moins denses


,P10,P40,P20,S76,M12,M30,B60
01,112.61,0.51,9.58,7.30,3.10,53.73,0.06
02,71.70,0.92,24.52,27.82,7.51,164.87,0.89
03,71.95,2.48,13.47,37.76,9.80,44.48,0.39
04,62.13,2.28,24.27,39.15,3.55,50.07,0.64
05,91.10,1.09,17.97,23.40,4.06,27.79,0.04
06,87.81,2.83,12.26,47.69,4.50,42.50,0.28
07,85.71,2.90,18.07,65.81,12.55,38.75,0.04
08,110.54,1.15,19.94,27.95,7.01,83.06,0.02
09,99.06,1.22,36.90,17.99,16.81,96.36,0.12
10,142.62,0.88,69.65,12.27,5.18,56.68,0.04


In [10]:
print("\n KPI 2 : Volume moyen en litre par type de contenant des 1000 ilpn les moins denses")

resultatKPI2 = pd.DataFrame(0, index=liste_contenant_interessant, columns=['Volume moyen (L)'])

for contenant in liste_contenant_interessant :
        liste = (rapport_stock_reserve[(rapport_stock_reserve["Contenant"] == contenant) & rapport_stock_reserve["Aire"].isin(liste_aire_interessante)]["m3"].values).tolist()
        liste = sorted(liste)
        liste = list(filter(lambda x: x != 0, liste))
        if len(liste) > 1000 :
            liste = liste[0:1000]
        moyenne = sum(liste)*1000 / max(len(liste), 1)
        resultatKPI2.loc[contenant] = moyenne

resultatKPI2=np.round(resultatKPI2,2)

resultatKPI2


 KPI 2 : Volume moyen en litre par type de contenant des 1000 ilpn les moins denses


,Volume moyen (L)
P10,100.90
P40,1.44
P20,19.17
S76,18.30
M12,5.90
M30,47.54
B60,0.08


In [11]:
# KPI 3 : le volume moyen occupé par alvéole dans l'entrepôt et par aire
print("\n KPI 2 : Volume alvéolaire")

resultat_volume_alveolaire = pd.DataFrame(0, index=liste_contenant_interessant, columns=['Volume'])
resultat_volume_alveolaire_total=pd.DataFrame(0, index=liste_contenant_interessant, columns=['Volume'])


for contenant in liste_contenant_interessant :
        liste = (rapport_stock_reserve[(rapport_stock_reserve["Contenant"] == contenant) & rapport_stock_reserve["Aire"].isin(liste_aire_interessante)]["m3"].values).tolist()
        liste_total = liste.copy()
        liste = sorted(liste)

        # On sélectionne uniquement les 1000 plus petits ILPNs :
        liste = list(filter(lambda x: x != 0, liste))
        if len(liste) > 1000 : 
            liste = liste[0:1000]

        moyenne = sum(liste)*1000 / max(len(liste), 1)
        moyenne_tot = sum(liste_total)*1000 / max(len(liste_total), 1)
        resultat_volume_alveolaire.loc[contenant] = moyenne
        resultat_volume_alveolaire_total.loc[contenant] = moyenne_tot

resultat_volume_alveolaire = np.round(resultat_volume_alveolaire, 2)
resultat_volume_alveolaire_total = np.round(resultat_volume_alveolaire_total, 2)

resultatKPI3 = pd.merge(resultat_volume_alveolaire, resultat_volume_alveolaire_total,left_index=True,right_index=True,how="inner")
#print(resultat)

# renommage des colonnes
nouveau_nom = {'Volume_x': 'Volume moyen des 1000 ilpn les moins denses en L', 'Volume_y': 'Volume moyen en L'}
resultatKPI3.rename(columns=nouveau_nom, inplace=True)


# on ajoute l'équivalent en volume alvéolaire en 
facteur = pd.DataFrame({"ctn":liste_contenant_interessant, "facteur": [1,1,1,1,4,2,15]})
resultatKPI3 = pd.merge(resultatKPI3, facteur, left_index=True, right_on="ctn", how="left")
resultatKPI3["équivalent en volume alvéolaire en L"] = resultatKPI3["facteur"] * resultatKPI3["Volume moyen en L"]
resultatKPI3 = resultatKPI3.set_index('ctn').drop('facteur', axis=1)

# on rajoute le nombre d'ILPNs
nbre_lpn = rapport_stock_reserve[(rapport_stock_reserve["Aire"].isin(liste_aire_interessante)) & \
        rapport_stock_reserve["Contenant"].isin(liste_contenant_interessant)].groupby('Contenant')['LPN'].nunique()
resultatKPI3 = pd.merge(resultatKPI3, nbre_lpn, left_index=True, right_index=True, how="left")

volume_standard_equiv = (resultatKPI3['équivalent en volume alvéolaire en L'] * resultatKPI3['LPN']).sum()/resultatKPI3['LPN'].sum()  
# Volume moyen par alvéole * multiplié par le nombre d'ILPN 
print("nombre total d'ILPN : ", resultatKPI3['LPN'].sum())
print("le volume alvéolaire équivalent est de ", np.round(volume_standard_equiv,1), "Litres")

kpi_3_1=np.round(volume_standard_equiv,1)

resultatKPI3=resultatKPI3.reset_index()
resultat_P40_asssimilée=resultatKPI3[resultatKPI3["ctn"].isin(liste_contenant_assimilé_P40)]
volume_std_equiv_P40_assimilé= (resultat_P40_asssimilée['équivalent en volume alvéolaire en L'] * resultat_P40_asssimilée['LPN']).sum()/resultat_P40_asssimilée['LPN'].sum()
#print("nombre total d'ILPN sur les P40 ou assimilé : ", resultat_P40_asssimilée['LPN'].values)
print("le volume alvéolaire équivalent sur de la P40 ou assimilé est de ",np.round(volume_std_equiv_P40_assimilé,1), "Litres")

print("le volume alvéolaire équivalent sur de la P40 ou assimilé est de ",np.round(volume_std_equiv_P40_assimilé,1), "Litres")

kpi_3_2=np.round(volume_std_equiv_P40_assimilé,1)

resultatKPI3 = resultatKPI3.set_index('ctn').drop('LPN', axis=1)
resultatKPI3





 KPI 2 : Volume alvéolaire
nombre total d'ILPN :  80342
le volume alvéolaire équivalent est de  401.3 Litres
le volume alvéolaire équivalent sur de la P40 ou assimilé est de  459.0 Litres
le volume alvéolaire équivalent sur de la P40 ou assimilé est de  459.0 Litres


,Volume moyen des 1000 ilpn les moins denses en L,Volume moyen en L,équivalent en volume alvéolaire en L
ctn,,,
P10,100.90,100.90,100.90
P40,1.44,484.61,484.61
P20,19.17,427.53,427.53
S76,18.30,302.66,302.66
M12,5.90,61.37,245.48
M30,47.54,142.83,285.66
B60,0.08,21.61,324.15


In [12]:
table_nombre_ILPN_par_reference_Puiseux = rapport_stock_reserve[rapport_stock_reserve["Aire"].isin(liste_aire_interessante)][['Référence', 'LPN']].groupby('Référence').count()
liste_Reference_ILPNs_unique = table_nombre_ILPN_par_reference_Puiseux[table_nombre_ILPN_par_reference_Puiseux['LPN'] == 1].index.tolist()
#print(len(liste_Reference_ILPNs_unique))

B=rapport_stock_reserve[(rapport_stock_reserve["m3"]<0.15) &  rapport_stock_reserve["Aire"].isin(liste_aire_interessante)& \
rapport_stock_reserve["Contenant"].isin(liste_contenant_assimilé_P40)]#.reindex(['Référence', 'Emplacement', 'Qté Standard LPN'], axis=1)

B=B[(B["Qté Standard LPN"]==B["Qté Stock"])]#[["Référence","Emplacement","Contenant","m3"]]
B=pd.merge(B,referentielPuiseux,left_on="Référence",right_on="id_pn_partnumber",how="inner")[["Référence","Emplacement","Categorie","Contenant","m3","LPN"]]

print("Toute aires confondues, il y a ",len(B)," emplacements occupés par une P40, dont la quantité est égale à l'ESP4, dont le volume est inférieur à 150 L ce qui représente",B["Référence"].nunique()," références")

resultatKPI4_1=len(B)
resultatKPI4_2=B["Référence"].nunique()

B.to_excel("donnees_sortie_pdf/P40 ou assimilé avec un volume ESP4 inférieur à 150 L.xlsx", index=True)

Toute aires confondues, il y a  158  emplacements occupés par une P40, dont la quantité est égale à l'ESP4, dont le volume est inférieur à 150 L ce qui représente 141  références


In [13]:
# KPI 5, calcule le taux d'occupation par typologie de contenant par cellule

occupation_reserve_filtre=occupation_reserve[
    (occupation_reserve["Qté LPN max"]>0) & 
    (occupation_reserve["Qté LPN max"]<16) & 
    pd.isnull(occupation_reserve["Blocage emplacement"]) &
    (occupation_reserve["Aire"].isin(liste_aire_interessante)) & 
    (occupation_reserve["Taille d'emplacement"].isin(liste_taille_emplacement_interessant))]

occupation_reserve_libre=pd.pivot_table(occupation_reserve_filtre, values='Qté LPN libre', index="Taille d'emplacement", columns='Zone', aggfunc=sum, fill_value=0)
occupation_reserve_tot=pd.pivot_table(occupation_reserve_filtre, values='Qté LPN max', index="Taille d'emplacement", columns='Zone', aggfunc=sum, fill_value=0)

occupation_reserve_libre=correspondance_taille_nom_emplacement.join(occupation_reserve_libre,on="Taille d'emplacement",how='right')
occupation_reserve_libre = occupation_reserve_libre.set_index('Nom').T

occupation_reserve_tot=correspondance_taille_nom_emplacement.join(occupation_reserve_tot,on="Taille d'emplacement",how='right')
occupation_reserve_tot = occupation_reserve_tot.set_index('Nom').T

occupation_reserve_libre=occupation_reserve_libre.groupby(level=0, axis=1).sum().drop("Taille d'emplacement")
occupation_reserve_tot=occupation_reserve_tot.groupby(level=0, axis=1).sum().drop("Taille d'emplacement")

taux_occupation = 1-occupation_reserve_libre/occupation_reserve_tot

resultatKPI5 = (taux_occupation * 100).astype(int).T

occupation_reserve_libre=occupation_reserve_libre.T.sum(axis=1)
occupation_reserve_tot=(occupation_reserve_tot.T.sum(axis=1))

#print(occupation_reserve_libre,occupation_reserve_tot)

taux_occ_glo= 1-occupation_reserve_libre/occupation_reserve_tot

#apv=np.round(pd.DataFrame(apv).T*100,1).drop('P10', axis=1)

taux_occ_glo=np.round(pd.DataFrame(taux_occ_glo)*100,2)

resultatKPI5_2=taux_occ_glo.T
resultatKPI5



,01,02,03,04,05,06,07,08,09,10,11
Nom,,,,,,,,,,,
B60,79,99,86,79,97,81,99,99,89,99,93
M12,88,91,95,95,95,98,88,96,80,84,95
M30,67,95,67,96,98,86,92,96,95,77,72
P10,91,98,100,94,100,100,98,97,101,100,93
P40,77,82,96,96,99,97,96,96,95,97,93


In [14]:
print(rapport_stock_actif.columns)
rapport_stock_actif_filtre=rapport_stock_actif[pd.isna(rapport_stock_actif['Type reference'])]
rapport_stock_actif_filtre=rapport_stock_actif_filtre[pd.isnull(rapport_stock_actif_filtre['Code blocage'])]
nb_casier_cellule=pd.pivot_table(rapport_stock_actif, values='Emplacement', index="Type emplacement", columns='Zone', aggfunc='count', fill_value=0)
nb_casier_cellule_vide=pd.pivot_table(rapport_stock_actif_filtre, values='Emplacement', index="Type emplacement", columns='Zone', aggfunc='count', fill_value=0)
resultatKPI6=nb_casier_cellule_vide
taux_occupation_actif=1-nb_casier_cellule_vide / nb_casier_cellule
taux_occupation_actif=np.round(pd.DataFrame(taux_occupation_actif)*100,2)
nb_casier_cellule_vide



Index(['Référence', 'Description', 'Volume Unit', 'Type reference',
       'Emplacement', 'Aire', 'Zone', 'Volume empl m3', 'Code blocage',
       'Max Casier', 'Qté stock', 'Qté allouée', 'Qté à arriver',
       'Type emplacement', 'Zone affect. prélèvement',
       'Pick location assignment types', 'Dénomination PLAT',
       'Type d'affectation', 'Zone détermination prélèvement',
       'Statut du flag réap'],
      dtype='object')


Zone,01,02,03,04,05,06,07,08,09,10,11,RF
Type emplacement,,,,,,,,,,,,
1.0 Palette,3,0,1,1,0,1,0,1,0,0,0,3
2.0 FIFO,103,11,0,0,0,146,135,78,15,0,4,0
2.1 LIFO,8,19,17,206,361,17,2,218,271,336,519,0
3.0 Tiroir,270,160,21,17,1144,0,0,833,620,2257,1308,0
3.1 Alveolaire 9L,0,0,5,0,67,0,0,1,84,14,30,0
3.2 Alveolaire 18L,0,0,0,0,26,0,0,0,0,0,0,0
4.0 Alveolaire 22L,33,14,1,44,34,133,123,32,3,41,1,0
4.1 Alveolaire 50L,22,13,7,16,105,15,62,39,24,27,38,0
4.2 Alveolaire 100L,11,19,5,10,15,15,4,1,5,11,14,0


In [15]:
# Calculer le nombre de casiers bloqués en HD et le nombre de double-casier

import pandas as pd

# Exemple de DataFrame
data = {'MaColonne': [1, 2, 2, 3, 3, 4, 5, 5]}
df = pd.DataFrame(data)

# Utilisez la méthode value_counts() pour compter les occurrences de chaque élément
occurrences = df['MaColonne'].value_counts()

# Sélectionnez les éléments qui apparaissent plus d'une fois
elements_recurrents = occurrences[occurrences > 1].index.tolist()

# Créez un nouveau DataFrame contenant uniquement les éléments récurrents
df_recurrents = df[df['MaColonne'].isin(elements_recurrents)]

# Affichez le DataFrame contenant les éléments récurrents
print(df_recurrents)

   MaColonne
1          2
2          2
3          3
4          3
6          5
7          5


In [16]:
# KPI 6

rapport_d = rapport_deselec
rapport_d = pd.merge(rapport_d, cause_acteur_deselec, left_on="GLOBAL_RSN_LBL", right_on="cause deselection", how="left")
rapport_d = rapport_d.sort_values(by="ordre de priorité").fillna(0)

RO = pd.DataFrame(rapport_d["acteur"].value_counts())
RO.rename(columns={'acteur': 'nombre de lignes non vaguées'}, inplace=True)
RO = pd.merge(RO, cause_acteur_deselec, left_index=True, right_on="acteur", how="right")[["acteur","nombre de lignes non vaguées","ordre de priorité"]]
RO = RO.drop_duplicates().sort_values(by="ordre de priorité").fillna(0)

NLV=nombre_lignes_non_vaguees.loc[0, 'Total Lines1']
nouvelle_colonne = [NLV]

for valeur in RO['nombre de lignes non vaguées'][0:len(RO)-1]:
    difference = nouvelle_colonne[-1]-valeur
    nouvelle_colonne.append(difference)

RO['nombre de ligne à vaguer'] = nouvelle_colonne
RO['RO'] = 1-RO['nombre de lignes non vaguées']/RO['nombre de ligne à vaguer']

ro_global=RO["RO"].prod()
ro_global = f"{ro_global * 100:.2f}%"

print("le RO global est de ", ro_global)
RO['RO']=RO['RO'].apply(lambda x: f"{x:.2%}")
RO = RO.set_index('acteur')
RO = RO.drop(['ordre de priorité'], axis=1)

resultatKPI7 = RO.copy()
ro_global_KPI6 = ro_global
resultatKPI7

le RO global est de  52.84%


,nombre de lignes non vaguées,nombre de ligne à vaguer,RO
acteur,,,
Debord,2005,6406,68.70%
Divers,6,4401,99.86%
Opération,738,4395,83.21%
Amenagement,256,3657,93.00%
Inventaire,16,3401,99.53%


In [17]:
def dataframe_vers_tableauPDF(dataframe, nom_index, transposé = False) :
    intermediaire = dataframe.rename_axis(nom_index)
    if transposé :
        intermediaire = intermediaire.T
    return([list(intermediaire.reset_index().columns)] + intermediaire.reset_index().values.tolist())


In [18]:

nombre_ref_réserve = rapport_stock_reserve[rapport_stock_reserve["Aire"].isin(liste_aire_interessante)]['Référence'].nunique()
nombre_ref_actif = rapport_stock_actif[rapport_stock_actif["Aire"].isin(liste_aire_interessante)]['Référence'].nunique()
print(nombre_ref_réserve,nombre_ref_actif)
liste_ref_actif = rapport_stock_actif[rapport_stock_actif["Aire"].isin(liste_aire_interessante)]['Référence'].drop_duplicates().tolist()
liste_ref_réserve=rapport_stock_reserve[rapport_stock_reserve["Aire"].isin(liste_aire_interessante)]['Référence'].drop_duplicates().tolist()
liste_ref_entrepot=liste_ref_actif+liste_ref_réserve
liste_ref_SE=rapport_stock_reserve[rapport_stock_reserve["Aire"]=="SE"]['Référence'].drop_duplicates().tolist()


#liste_ref_SE_uniquement=[element for element in liste_ref_SE if element not in liste_ref_entrepot]

liste_ref_SE_uniquement=list(set(liste_ref_SE)-set(liste_ref_entrepot))
nombre_ref_entrepôt=len(set(liste_ref_entrepot))

taux_dispo=np.round((nombre_ref_entrepôt/(nombre_ref_entrepôt+len(liste_ref_SE_uniquement)))*100,2)
taux_dispo='{:.2f} %'.format(taux_dispo)

nombre_ref_actif='{:,.0f}'.format(nombre_ref_actif).replace(',', ' ') #correction du format pour avoir un espace entre chaque millier
nombre_ref_entrepôt= '{:,.0f}'.format(nombre_ref_entrepôt).replace(',', ' ')
nombre_ref_réserve='{:,.0f}'.format(nombre_ref_réserve).replace(',', ' ')
nombre_ref_SE_uniquement='{:,.0f}'.format(len(liste_ref_SE_uniquement)).replace(',', ' ')

synthèse_taux_dispo = { 
    "KPI" : ["Nombre de ref à l'actif", "Nombre de ref en réserve", "Nombre de ref dans les murs de Puiseux", "nombre de ref en SE uniquement", "taux de dispo du référentiel"],
    "valeur" : [nombre_ref_actif, nombre_ref_réserve,nombre_ref_entrepôt, nombre_ref_SE_uniquement, taux_dispo ]
}
synthèse_taux_dispo=pd.DataFrame(synthèse_taux_dispo).set_index("KPI")

resultatKPI8 = synthèse_taux_dispo

synthèse_taux_dispo

57723 52451


,valeur
KPI,
Nombre de ref à l'actif,52 451
Nombre de ref en réserve,57 723
Nombre de ref dans les murs de Puiseux,93 267
nombre de ref en SE uniquement,11 766
taux de dispo du référentiel,88.80 %


In [19]:
tableKPI1 = Table(dataframe_vers_tableauPDF(resultatKPI1, 'Aire'))
tableKPI2 = Table(dataframe_vers_tableauPDF(resultatKPI2, 'Contenant', transposé=True))
tableKPI3 = Table(dataframe_vers_tableauPDF(resultatKPI3, 'Aire', transposé=True))
tableKPI5 = Table(dataframe_vers_tableauPDF(resultatKPI5, 'Cellule'))
tableKPI5_2 = Table(dataframe_vers_tableauPDF(resultatKPI5_2, 'Cellule'))
tableKPI6 = Table(dataframe_vers_tableauPDF(resultatKPI6, 'Acteur'))
tableKPI7 = Table(dataframe_vers_tableauPDF(resultatKPI7, 'Acteur'))
tableKPI8 = Table(dataframe_vers_tableauPDF(resultatKPI8, 'Acteur'))

# Appliquer un style au tableau
style = TableStyle([
    ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
    ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
    ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
    ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
    ('BACKGROUND', (0, 0), (0, -1), colors.grey),
    ('BACKGROUND', (0, 0), (0, 0), colors.white),
    ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
    ('TEXTCOLOR', (0, 0), (0, -1), colors.whitesmoke),
    ('TEXTCOLOR', (0, 0), (0, 0), colors.white),
    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
    ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
    ('FONTNAME', (0, 0), (0, -1), 'Helvetica-Bold'),
    ('BOTTOMPADDING', (0, 0), (0, -1), 12),
    ('GRID', (1, 0), (-1, -1), 1, colors.black), # Ignorer la première colonne et la première ligne pour les bordures
    ('GRID', (0, 1), (-1, -1), 1, colors.black), # Ignorer la première colonne et la première ligne pour les bordures

])
tableKPI1.setStyle(style)
tableKPI2.setStyle(style)
tableKPI3.setStyle(style)
tableKPI5.setStyle(style)
tableKPI5_2.setStyle(style)
tableKPI6.setStyle(style)
tableKPI7.setStyle(style)
tableKPI8.setStyle(style)

In [20]:
#from IPython.display import HTML
#
## Définir la valeur seuil
#seuil = 95
#
## Fonction pour colorer en rouge les valeurs supérieures au seuil
#def color_red(val):
#    color = 'red' if val > seuil else 'black'
#    return f'color: {color}'
#
## Appliquer la fonction aux valeurs du DataFrame
#resultatKPI5 = resultatKPI5.style.applymap(color_red)
#
## Afficher le DataFrame stylisé avec les couleurs
#HTML(resultatKPI5.render())

In [ ]:
def fonction_ajout_element(paragraph_text) :
    resultat = []
    if paragraph_text == "KPI 1 : Volume moyen en litre des 100 ILPNs les moins volumineux par type de contenant et par aire" :
        texte = "Raisonnement : uniquement sur Puiseux, pour chaque type de contenant principal et chaque aire de travail, \
            voici le volume moyen en litre des 100 ILPNs les moins volumineux de ce type de contenant et de cette aire."
        resultat.append(Paragraph(texte, styles['Normal']))
        resultat.append(Spacer(1, 12))
        resultat.append(tableKPI1)
    elif paragraph_text == "KPI 2 : Volume moyen en litre des 1000 ILPNs les moins volumineux" :
        texte = "Raisonnement : KPI en complémentaire du premier, uniquement sur Puiseux ; pour chaque type de contenant principal, \
            voici le volume moyen en litre des 1000 ILPNs les moins volumineux de ce type de contenant."
        resultat.append(Paragraph(texte, styles['Normal']))
        resultat.append(Spacer(1, 12))
        resultat.append(tableKPI2)
    elif paragraph_text == "KPI 3 : Volume moyen occupé par alvéole" :
        texte = f"Deux indicateur sont analysés <b>Le volume alvéolaire moyen est de {kpi_3_1} L, et le volume alvéolaire moyen pour des alvéoles mono-ref est de {kpi_3_2} L (hors P10) </b>."
        resultat.append(Paragraph(texte, styles['Normal']))
        resultat.append(Spacer(1, 12))
        #resultat.append(tableKPI3)
    elif paragraph_text == "KPI 4 : Référence conditionné en P40 ou assimilé dont le volume de la quantité standard est inférieur à 150 L" :
        texte = f"Raisonnement : sur l'entrepôt de Puiseux uniquement, <b>il y a {resultatKPI4_1} emplacements P40 ou assimilé dont la quantité est égale à l'ESP4 et dont le volume est inférieur à 150 L.</b>."
        resultat.append(Paragraph(texte, styles['Normal']))
        resultat.append(Spacer(1, 12))
    elif paragraph_text == "KPI 5 : Taux d'occupation en réserve par typologie de contenant et par cellule" :
        texte = f"Raisonnement : sur l'entrepôt de Puiseux uniquement, voici le taux d'occupation en réserve détaillé par typologie de contenant."
        resultat.append(Paragraph(texte, styles['Normal']))
        resultat.append(Spacer(1, 12))
        resultat.append(tableKPI5)
        resultat.append(Spacer(1, 12))
        texte = f"Taux d'occupation global par typologie de contenant de contenant"
        resultat.append(Spacer(1, 12))
        resultat.append(Paragraph(texte, styles['Normal']))
        resultat.append(Spacer(1, 12))
        resultat.append(tableKPI5_2)
    elif paragraph_text == "KPI 6 : Casiers actifs disponibles" :
        texte = f"Nombre de casiers disponible par cellule et par type"
        resultat.append(Paragraph(texte, styles['Normal']))
        resultat.append(Spacer(1, 12))
        resultat.append(tableKPI6)
    elif paragraph_text == "KPI 7 : RO global et détaillé par unité" :
        texte = f"Raisonnement : sur l'entrepôt de Puiseux uniquement, <b>le rendement opérationnel global est de {ro_global_KPI6}</b>. Celui-ci est déterminé à partir des RO de chaque unité, en cascade selon l'ordre de ces unités"
        resultat.append(Paragraph(texte, styles['Normal']))
        resultat.append(Spacer(1, 12))
        resultat.append(tableKPI7)
    elif paragraph_text == "KPI 8 : évolution du référentiel Puiseux" :
        texte = f"Ci-dessous l'analyse du taux de disponibilité du référentiel Puiseux, avec stock"
        resultat.append(Spacer(1, 12))
        resultat.append(tableKPI8)
    return(resultat)
def fonction_ajout_element(paragraph_text) :
    resultat = []
    if paragraph_text == "KPI 1 : Volume moyen en litre des 100 ILPNs les moins volumineux par type de contenant et par aire" :
        texte = "Raisonnement : uniquement sur Puiseux, pour chaque type de contenant principal et chaque aire de travail, \
            voici le volume moyen en litre des 100 ILPNs les moins volumineux de ce type de contenant et de cette aire."
        resultat.append(Paragraph(texte, styles['Normal']))
        resultat.append(Spacer(1, 12))
        resultat.append(tableKPI1)
    elif paragraph_text == "KPI 2 : Volume moyen en litre des 1000 ILPNs les moins volumineux" :
        texte = "Raisonnement : KPI en complémentaire du premier, uniquement sur Puiseux ; pour chaque type de contenant principal, \
            voici le volume moyen en litre des 1000 ILPNs les moins volumineux de ce type de contenant."
        resultat.append(Paragraph(texte, styles['Normal']))
        resultat.append(Spacer(1, 12))
        resultat.append(tableKPI2)
    elif paragraph_text == "KPI 3 : Volume moyen occupé par alvéole" :
        texte = f"Deux indicateur sont analysés <b>Le volume alvéolaire moyen est de {kpi_3_1} L, et le volume alvéolaire moyen pour des alvéoles mono-ref est de {kpi_3_2} L (hors P10) </b>."
        resultat.append(Paragraph(texte, styles['Normal']))
        resultat.append(Spacer(1, 12))
        #resultat.append(tableKPI3)
    elif paragraph_text == "KPI 4 : Référence conditionné en P40 ou assimilé dont le volume de la quantité standard est inférieur à 150 L" :
        texte = f"Raisonnement : sur l'entrepôt de Puiseux uniquement, <b>il y a {resultatKPI4_1} emplacements P40 ou assimilé dont la quantité est égale à l'ESP4 et dont le volume est inférieur à 150 L.</b>."
        resultat.append(Paragraph(texte, styles['Normal']))
        resultat.append(Spacer(1, 12))
    elif paragraph_text == "KPI 5 : Taux d'occupation en réserve par typologie de contenant et par cellule" :
        texte = f"Raisonnement : sur l'entrepôt de Puiseux uniquement, voici le taux d'occupation en réserve détaillé par typologie de contenant."
        resultat.append(Paragraph(texte, styles['Normal']))
        resultat.append(Spacer(1, 12))
        resultat.append(tableKPI5)
        resultat.append(Spacer(1, 12))
        texte = f"Taux d'occupation global par typologie de contenant de contenant"
        resultat.append(Spacer(1, 12))
        resultat.append(Paragraph(texte, styles['Normal']))
        resultat.append(Spacer(1, 12))
        resultat.append(tableKPI5_2)
    elif paragraph_text == "KPI 6 : Casiers actifs disponibles" :
        texte = f"Nombre de casiers disponible par cellule et par type"
        resultat.append(Paragraph(texte, styles['Normal']))
        resultat.append(Spacer(1, 12))
        resultat.append(tableKPI6)
    elif paragraph_text == "KPI 7 : RO global et détaillé par unité" :
        texte = f"Raisonnement : sur l'entrepôt de Puiseux uniquement, <b>le rendement opérationnel global est de {ro_global_KPI6}</b>. Celui-ci est déterminé à partir des RO de chaque unité, en cascade selon l'ordre de ces unités"
        resultat.append(Paragraph(texte, styles['Normal']))
        resultat.append(Spacer(1, 12))
        resultat.append(tableKPI7)
    elif paragraph_text == "KPI 8 : évolution du référentiel Puiseux" :
        texte = f"Ci-dessous l'analyse du taux de disponibilité du référentiel Puiseux, avec stock"
        resultat.append(Spacer(1, 12))
        resultat.append(tableKPI8)
    return(resultat)


In [22]:
# Styles de paragraphe prédéfinis
styles = getSampleStyleSheet()

# Données pour les sections et les paragraphes
sections = [
    
    {"title": "KPI de réserve", \
        "content": ["KPI 1 : Volume moyen en litre des 100 ILPNs les moins volumineux par type de contenant et par aire", \
            "KPI 2 : Volume moyen en litre des 1000 ILPNs les moins volumineux", \
            "KPI 3 : Volume moyen occupé par alvéole",
            "KPI 4 : Référence conditionné en P40 ou assimilé dont le volume de la quantité standard est inférieur à 150 L", 
            "KPI 5 : Taux d'occupation en réserve par typologie de contenant et par cellule",
            ]},
    {"title": "KPI sur l'actif", "content": ["KPI 6 : Casiers actifs disponibles"]},
    {"title": "KPI de Rendement Opérationnel", "content": ["KPI 7 : RO global et détaillé par unité"]},

    {"title": "KPI généraux", "content": ["KPI 8 : évolution du référentiel Puiseux"]},
    # Ajoutez plus de sections au besoin
]

doc = SimpleDocTemplate(chemin_sortie_pdf, pagesize=letter)

# Liste pour stocker les éléments du document
elements = []

date = datetime.today().strftime('%y_%m_%d')
title = "Rapport de KPI, pôle performance, du " + date
elements.append(Paragraph(title, styles['Title']))
elements.append(Spacer(1, 12))
elements.append(Paragraph("Ci-dessous une fiche staff récapitulant les indicateurs de l'UET méthodes", styles['Normal']))
elements.append(Spacer(1, 12))

# Parcourir les sections et les paragraphes
for section in sections:
    # Ajouter le titre de la section au document
    elements.append(Paragraph(section["title"], styles['Heading1']))
    
    # Ajouter chaque paragraphe de la section au document
    for paragraph_text in section["content"]:
        paragraph = Paragraph(paragraph_text, styles["Heading2"])
        elements.append(paragraph)
        elements.append(Spacer(1, 12))
        ajouts = fonction_ajout_element(paragraph_text)
        for a in ajouts :
            elements.append(a)
        
    # Ajouter un espace entre les sections
    #elements.append(Spacer(1, 12))
    # Ajouter un saut de page entre les sections
    elements.append(PageBreak())


# Construire le document
doc.build(elements)

print(f"Le fichier {chemin_sortie_pdf} a été créé avec succès.")

Le fichier donnees_sortie_pdf/24_03_07_fiche_staff.pdf a été créé avec succès.
